# 函数
---

### 匿名函数捕获变量值

In [1]:
# lambda 表达式是在执行时才取自由变量的值
funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0), end=' ')

4 4 4 4 4 

In [2]:
# 如果想在定义时捕获变量值，需要将参数值定义成默认参数
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0), end=' ')

0 1 2 3 4 

---

### 减少可调用对象的参数个数

In [3]:
# partial 函数允许给多个参数设置固定的值，减少之后被调用时的参数个数
from functools import partial

def spam(a, b, c, d):
    print(a, b, c, d)

s1 = partial(spam, 1) # a=1
s2 = partial(spam, d=42)
s3 = partial(spam, 21, 26, d=42)

In [4]:
s1(2, 3, 4)

1 2 3 4


In [5]:
s2(1, 2, 3)

1 2 3 42


In [6]:
s3(39)

21 26 39 42


---

### 将单方法的类改写为函数

方法一

In [7]:
class UrlTemplate:
    def __init__(self, template):
        self.template = template

    def join(self, **kwargs):
        return self.template.format_map(kwargs)

In [8]:
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')

In [9]:
yahoo.join(names='IBM,AAPL,FB', fields='sl1c1v')

'http://finance.yahoo.com/d/quotes.csv?s=IBM,AAPL,FB&f=sl1c1v'

方法二，利用闭包函数实现

In [10]:
def url_template(template):
    def opener(**kwargs):
        return template.format_map(kwargs)
    return opener

In [11]:
yahoo = url_template('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')

In [12]:
yahoo(names='IBM,AAPL,FB', fields='sl1c1v')

'http://finance.yahoo.com/d/quotes.csv?s=IBM,AAPL,FB&f=sl1c1v'

---

### 通过回调函数增加额外信息

print_result 函数只能接受一个参数，假设现在需要为结果输出增加序号

In [13]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y

In [14]:
apply_async(add, ('hello', 'world'), callback=print_result)

Got: helloworld


 方法一

In [15]:
class ResultHandler:

    def __init__(self):
        self.sequence = 0

    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))

In [16]:
r = ResultHandler()

In [17]:
apply_async(add, (2, 3), callback=r.handler)

[1] Got: 5


In [18]:
apply_async(add, ('hello', 'world'), callback=r.handler)

[2] Got: helloworld


方法二

In [19]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

In [20]:
handler = make_handler()

In [21]:
apply_async(add, (2, 3), callback=handler)

[1] Got: 5


In [22]:
apply_async(add, ('hello', 'world'), callback=handler)

[2] Got: helloworld


方法三

In [23]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

In [24]:
handler = make_handler()

In [25]:
# 这里必须先用 next 函数，否则会报错
next(handler)

In [26]:
apply_async(add, (2, 3), callback=handler.send)

[1] Got: 5


In [27]:
apply_async(add, ('hello', 'world'), callback=handler.send)

[2] Got: helloworld


---

### 内联回调函数

使用装饰器实现内联回调（InlineCallback），  
可以让代码仅添加 yield 就实现回调，避免弄乱原本的执行序列

In [28]:
from queue import Queue
from functools import wraps

def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

In [29]:
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [30]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


---

### 访问闭包中定义的变量

In [31]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n =', n)

    # Accessor methods for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

f = sample()

In [32]:
f()

n = 0


In [33]:
f.set_n(10)

In [34]:
f()

n = 10


In [35]:
f.get_n()

10